## Studying tunning of spark

1.) __spark.cores.max__ is the maximum number of core.

2.) __spark.executor.cores__ property controls the number of concurrent tasks an executor can run. (__spark.executor.cores=5__ means that each executor can run a maximum of five tasks at the same time)

3.) __spark.executor.memory__ The memory property impacts the amount of data Spark can cache, as well as the maximum sizes of the shuffle data structures used for grouping, aggregations, and joins.

4.) __spark.executor.instances__ configuration property control the number of executors requested. (Spark 1.3, you will be able to avoid setting this property by turning on dynamic allocation with the __spark.dynamicAllocation.enabled__ property. Dynamic allocation enables a Spark application to request executors when there is a backlog of pending tasks and free up executors when idle.)



Notice:

1.) high number of core per executor will lead to bad HDFS I/O throughput.

In [ ]:
import pyspark.sql
session = pyspark.sql.SparkSession.builder \
    .master('spark://10.64.22.215:7077') \
    .appName('Zpeak_Nanoaod-SPARK-Histbook') \
    .config('spark.jars.packages','org.diana-hep:spark-root_2.11:0.1.16,org.diana-hep:histogrammar-sparksql_2.11:1.0.4') \
    .config('spark.driver.extraClassPath','/opt/hadoop/share/hadoop/common/lib/EOSfs.jar') \
    .config('spark.executor.extraClassPath','/opt/hadoop/share/hadoop/common/lib/EOSfs.jar') \
    .config('spark.sql.caseSensitive','true') \
    .config('spark.serializer','org.apache.spark.serializer.KryoSerializer') \
    .config('spark.cores.max','40') \
    .config('spark.executor.instances','11') \
    .config('spark.executor.cores','3') \
    .config('spark.executor.memory','1g') \
    .getOrCreate()
    
sqlContext = session
print 'Spark version: ',sqlContext.version
print 'SparkSQL sesssion created'

In [ ]:
from pyspark.sql.functions import lit
from samples import *

DFList = [] 

for s in samples:
    print 'Loading {0} sample from EOS file'.format(s) 
    dsPath = "root://eospublic.cern.ch//eos/opstest/cmspd-bigdata/"+samples[s]['filename']    
    tempDF = sqlContext.read \
                .format("org.dianahep.sparkroot") \
                .option("tree", "Events") \
                .load(dsPath)\
                .withColumn("pseudoweight", lit(samples[s]['weight'])) \
                .withColumn("sample", lit(s))                
    DFList.append(tempDF)

In [ ]:
# Define interesting attributes to be selected
columns = [
    ### MUON
    'Muon_pt',
    'Muon_eta',
    'Muon_phi',
    'Muon_mass',
    'Muon_charge',
    'Muon_mediumId',
    'Muon_softId',
    'Muon_tightId',
    'nMuon',
    ### SAMPLE
    'sample',
    ### Jet
    'nJet',
    'Jet_pt',
    'Jet_eta',
    'Jet_phi',
    'Jet_mass',
    'Jet_bReg',
    ### Weight
    'pseudoweight',
]

# Select columns from dataframe
DF = DFList[0].select(columns)

# Merge all dataset into a single dataframe
for df_ in DFList[1:]:
    DF = DF.union(df_.select(columns))

In [ ]:
print 'Partition allocated for Dataframe:',DF.rdd.getNumPartitions(), 'partition'
print 'Partition allocated for Dataframe reported from executors (JVM):',sqlContext._jsc.sc().getExecutorMemoryStatus().size(), 'partition'
print 'Default number of partition (defaultParallelism) = ',sqlContext._jsc.sc().defaultParallelism()

In [ ]:
sqlContext._jsc._conf.getAll()